In [440]:
# Constants
ALPH = "абвгдежзийклмнопрстуфхцчшщъыьэюя"
PROBS = [0.0792, 0.0171, 0.0433, 0.0174, 0.0305, 0.0841, 0.0105, 0.0175,
         0.0683, 0.0112, 0.0336, 0.0500, 0.0326, 0.0672, 0.1108, 0.0281,
         0.0445, 0.0533, 0.0618, 0.0280, 0.0019, 0.0089, 0.0036, 0.0147,
         0.0081, 0.0037, 0.0002, 0.0196, 0.0192, 0.0038, 0.0061, 0.0213]
ALPH_SIZE = len(ALPH)
RING = {}
for char in ALPH:
    pos = ALPH.index(char)
    RING[char] = pos
EXPECTED_I = sum([(p ** 2) for p in PROBS])
I0 = 1.0 / ALPH_SIZE
FILE_NAME = "text.txt"

In [441]:
def transform_symbol(_c):
    if 'а' <= _c and _c <= 'я':
        return _c
    elif _c <= 'Я' and _c >= 'А':
        return _c.lower()
    elif _c == 'Ё' or _c == 'ё':
        return 'е'
    else:
        return ""
    
def preprocess_text(_text):
    text_formatted = ""
    # Change symbols according to requirements
    for c in _text:
        text_formatted += transform_symbol(c)
# Треба функція для додавання 
    # Remove consequtive spaces
    text_formatted = ' '.join(text_formatted.split())
    return text_formatted

def read_text(filename):
    f = open(filename, "r", encoding='utf-8')
    text = f.read()
    f.close()
    return preprocess_text(text)

In [442]:
def transform_symbol(_c):
    if 'а' <= _c and _c <= 'я':
        return _c
    elif _c <= 'Я' and _c >= 'А':
        return _c.lower()
    elif _c == 'Ё' or _c == 'ё': 
        return 'е'
    else:
        return ""
    
def preprocess_text(_text):
    text_formatted = ""
    # Change symbols according to requirements
    for c in _text:
        text_formatted += transform_symbol(c)
    # Remove consequtive spaces
    text_formatted = ' '.join(text_formatted.split())
    return text_formatted

def read_text(filename):
    f = open(filename, "r", encoding='utf-8')
    text = f.read()
    f.close()
    return preprocess_text(text)

In [443]:
def coincidence_index(text: list) -> int:
    sum = 0
    for c in ALPH:
        occurances = text.count(c)
        sum += occurances * (occurances - 1)
    return sum / (len(text) * (len(text) - 1))

def divide_into_blocks(text: list, r: int):
    res = []
    for i in range(r):
        block = text[i::r] 
        res.append(block)
    # print(res)
    return res

def coinc_idx_for_blocks(text: list, r: int):
    idxs = []
    blocks = divide_into_blocks(text, r)
    for i in range(r):
        idxs.append(coincidence_index(blocks[i]))
    return idxs

def summary_af_idx_for_diff_r(text: list):
    idx_table = {}
    # if we assume that len(KEY) divides len(text) -> itr through divisors
    # also we decided to make unified solution (check also if key length > 30)
    itr_range = range(2, int(len(text) / 2))     
    for r in itr_range:
        idx_table[r] = coinc_idx_for_blocks(text, r)
    return idx_table

In [444]:
import numpy.linalg as linalg
# Instead of comparing average idx diff, we measure vector distance and compare
def compare_idx_vectors(left: list, right: list) -> bool:
    left_dist = linalg.norm([EXPECTED_I - idx for idx in left])
    right_dist = linalg.norm([EXPECTED_I - idx for idx in right])
    return left_dist <= right_dist

def get_best_r(text: list):
    best_r = 2
    idx_map = summary_af_idx_for_diff_r(text)
    for (r, idx) in idx_map.items():
        if compare_idx_vectors(idx, idx_map[best_r]):
            best_r = r
            print(f"{r}: {sum(idx) / len(idx)}")
    return best_r


### Метод частот

In [445]:
def crack_key_freq(text, r) -> str:
    key = ""
    blocks = divide_into_blocks(text, r)
    for i in range(r):
        y = RING[max(blocks[i], key = blocks[i].count)]
        x = PROBS.index(max(PROBS))
        k_i = (y - x) % ALPH_SIZE
        key += ALPH[k_i]
    return key

In [446]:
def crack_key_freq_mod(text, r, found_pos=[], iteration=0, prev_key="") -> str:
    key = ""
    blocks = divide_into_blocks(text, r)
    for i in range(r):
        if i not in found_pos:
            y = RING[max(blocks[i], key = blocks[i].count)]
            x = PROBS.index(sorted(PROBS)[-(iteration + 1)])
            k_i = (y - x) % ALPH_SIZE
            key += ALPH[k_i]
        else:
            key += prev_key[i]
    return key

### Метод $M_i(g)$

In [447]:
def crack_key_Mi(text, r) -> str:
    key = ""
    blocks = divide_into_blocks(text, r)
    for i in range(r):
        k_i = 0
        M_max = 0
        for g in range(ALPH_SIZE):
            try_M = 0
            for t in range(ALPH_SIZE):
                try_M += PROBS[t] * blocks[i].count(ALPH[(t + g) % ALPH_SIZE])
            if try_M > M_max:
                M_max = try_M
                k_i = g
        key += ALPH[k_i]
    return key


In [448]:
def success_idxes(right_key, to_improve, r):
    res = []
    for i in range(r):
        if right_key[i] == to_improve[i]:
            res.append(i)
    return res

# This is to complete frequency method
# Kind of simulation of checking artifacts in text manualy :)
def crack_freq_iterable(text: str, r: int, expected_key: str):
    key = crack_key_freq_mod(text, r)
    for i in range(1, ALPH_SIZE):
        print(f"Freq key (itr {i - 1}): {key}")
        if key != expected_key:
            found = success_idxes(expected_key, key, r)
            key = crack_key_freq_mod(text, r, found_pos=found, iteration=i, prev_key=key)
        else:
            break
    return key



In [449]:
def decrypt_text(text, key):
    res = ""
    print(len(text))
    r = len(key)
    for i in range(0, len(text), r):
        for j in range(r):
            if i + j == len(text):
                break
            id = (RING[text[i + j]] - RING[key[j]]) % ALPH_SIZE
            res += ALPH[id]

    return res

In [450]:
text = read_text(FILE_NAME)
r = get_best_r(text)
keyMi = crack_key_Mi(text, r)
print(f"Mi key: {keyMi}")
key = crack_freq_iterable(text, r, keyMi)
print(f"Freq key result: {key}")

decrypt_text(text, keyMi)

2: 0.03709682620655367
16: 0.05539766505382552
Mi key: делолисоборотней
Freq key (itr 0): декелисоборойдей
Freq key (itr 1): деуолисоборотней
Freq key (itr 2): дешолисоборотней
Freq key (itr 3): деролисоборотней
Freq key (itr 4): делолисоборотней
Freq key result: делолисоборотней
5614


'понятноеделокультурунасильновчеловеканевоткнешьвордусиэтудовольногрустнуюистинузналинаверноелучшечемгдебытонибыловмирекультурностьпреждевсегоусилиеиежелионосызмальстванесделалосьчеловекусвычнымдажевнутреннепотребнымоттоготомногочисленныеподразделенияпалатыцеремонийиуделяютстольковниманиядетямособеннодетямтехктонаселяетхутуныпотомужобычнаяленостьлюдскаяслужитемупочтинеодолимымпрепятствиемнанеобъятныхпросторахимпериивстречаетсяещенемалолюдейкоторымпокакимтолишьбуддазнаеткакимпричинамтакинесталоинтереснымничтоглавноенисветозарныевысотыдухавеликихрелигийивечныйпоисксмыслажизниземнойпитающийистинноеискусствониголовокружительныебезднынакраюкоихвечнопребываетнастилающаянаднимиобщепроходимыегатинауканихотябычистоепросторноесостоятельноеидобродетельноежитьестольестественноедлябольшинстваордусскихподданныхчтогрехатаитьхутунынаселеныбыливосновномварварамииневобычномпониманииэтогословаисстариобозначавшеголюдейинойнеордусскойкультурыаскореевтомегозначениикотороестольжедавносделалосьобычнымвевропел